In [1]:
import numpy as np
import pandas as pd
import statsmodels.stats.multitest as smm

### P Value Analysis of Slow Method

In [2]:
pValFile = 'data/output_parallel.tsv'
pValDf = pd.read_csv(pValFile, sep='\t', index_col=0)
pValDf = np.abs(pValDf)
pValDf.head()

,A2M,AATF,ABCA1,ABCA3,ABCB1,ABCG1,ABCG5,ABCG8,ABL1,ACACA,...,ZNF350,ZNF382,ZNF383,ZNF385A,ZNF444,ZNF467,ZNF638,ZNF76,ZNRD1,ZYX
midpoint_AAACCTGCACCCAGTG-1,NaN,NaN,NaN,1.000000e-07,1.000000e-07,NaN,NaN,NaN,5.659419e-01,NaN,...,NaN,1.963683e-01,NaN,4.533515e-01,NaN,NaN,NaN,NaN,NaN,NaN
midpoint_AAACCTGCAGCTTAAC-1,0.677111,NaN,NaN,NaN,1.000000e-07,NaN,NaN,NaN,1.000000e-07,NaN,...,NaN,5.918660e-01,NaN,1.000000e-07,NaN,NaN,NaN,5.707351e-01,NaN,NaN
midpoint_AAACCTGGTGTGCGTC-1,0.893910,2.113175e-01,NaN,1.000000e-07,3.052760e-02,NaN,NaN,NaN,1.000000e-07,NaN,...,2.450706e-01,1.000000e-07,1.000000e-07,1.000000e-07,NaN,NaN,NaN,5.635126e-01,NaN,1.000000e-07
midpoint_AAACCTGGTTGAACTC-1,0.974321,NaN,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,9.781122e-01,1.000000e-07,...,NaN,6.536430e-02,1.842525e-01,2.382358e-01,NaN,NaN,NaN,1.000000e-07,0.765877,NaN
midpoint_AAACGGGAGGATGGTC-1,0.466762,1.000000e-07,1.000000e-07,NaN,5.493695e-01,1.000000e-07,1.000000e-07,1.000000e-07,2.410608e-01,NaN,...,1.000000e-07,1.000000e-07,1.000000e-07,2.973550e-02,NaN,NaN,NaN,1.000000e-07,NaN,NaN


In [3]:
adjustedPVals = pd.DataFrame(columns=pValDf.columns, index=pValDf.index)
actRej = pd.DataFrame(columns=pValDf.columns, index=pValDf.index)


for (tf, pVals) in pValDf.iteritems():
    pVals = pVals.dropna()
    pV_df = pd.DataFrame(index=pVals.index, columns=['pVal'], data=pVals.values)
    pV_df.sort_values(by='pVal', inplace=True)

    reject, adj_p_val = smm.fdrcorrection(pV_df['pVal'].values, method='i', is_sorted=True, alpha=0.1)
    pV_df[tf] = adj_p_val

    # Insert pV_df into adjustedPVals by column by index
    adjustedPVals.loc[pV_df.index.values, tf] = pV_df[tf].values
    actRej.loc[pV_df.index.values, tf] = reject

adjustedPVals.to_csv('data/adjustedPValsSlw5k.tsv', sep='\t')
actRej.to_csv('data/actRejSlw5k.tsv', sep='\t')

In [4]:
aRS_df = pd.read_csv('data/actRejSlw5k.tsv', sep='\t', index_col=0)
hashSX0 = aRS_df.sum(axis=0)
hashSX1 = aRS_df.sum(axis=1).to_frame()

/home/kisanthapa/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (16,23,34,42,65,76,89,91,93,102,148,158,175,192,195,201,204,206,207,208,221,247,248,250,256,291,294,297,299,302,307,319,320,339,340,344,352,374,382,417,433,443,444,445,450,453,461,469,475,477,481,483,484,514,521,523,524,528,534,537,538,539,540,571,592,602,603,622,625,627,635,636,677,690,701,702,704,707,711,729,730,731,732,733,734,735,736,737,738,739,740,741,742,748,756,760,761,763,781,782,790,797,811,813,819,822,828,829,831,833,841,843,854,855,856,858,895,927,930,981,987,1008,1011,1024,1026,1035,1036,1039,1040,1053,1059,1062,1069,1077,1078,1080,1081,1082,1095,1104,1117,1120,1122,1123,1125,1127,1137,1140,1141,1143,1147,1148,1152,1165,1167,1171,1179,1214,1239,1242,1243,1252,1265,1267,1273,1280,1282,1287,1290,1317,1320,1329,1331,1349,1351,1352,1353,1354,1363,1391,1393,1394,1402,1411,1422,1425,1426,1429,1432,1443,1452,1455,1463,1465,1474,1476,1485,1490,1504,1507,1514,1533,153